# Social Security file Validator

## Input document type and quarter

Valid document type: "core", "dmfa", "dmfareq"<br>Valid quarter: 20223, 20222, 20221

In [ ]:
docType = input()
quarter = input()

## Input file to validate

In [ ]:
filename = input() # ex: valid_dmfa.ttl, invalid_dmfa.ttl

#Check that file exist, crash if does not exist
file = open(filename)
file.close()

## Input triple store endpoint

In [ ]:
endpoint = input() #ex: http://localhost:3030/socialsecurity

## Validation

In [ ]:
# SPARQL endpoint setup
from SPARQLWrapper import SPARQLWrapper, TURTLE
sparql = SPARQLWrapper(endpoint)
sparql.setReturnFormat(TURTLE)

Creating data graph


In [ ]:
# Copy input file
import shutil
datagraphFilename = filename.split(".")[0] + "_datagraph_tmp.ttl"
shutil.copy(filename, datagraphFilename)

# Fetch and append annexes
FETCH_ANNEXES_QUERY = """
CONSTRUCT{
	?s ?p ?o.
}
FROM <http://kg.socialsecurity.be/resource/annexes/2/>
FROM <http://kg.socialsecurity.be/resource/annexes/4/>
FROM <http://kg.socialsecurity.be/resource/annexes/6/>
FROM <http://kg.socialsecurity.be/resource/annexes/7/>
FROM <http://kg.socialsecurity.be/resource/annexes/8/>
FROM <http://kg.socialsecurity.be/resource/annexes/9/>
FROM <http://kg.socialsecurity.be/resource/annexes/10/>
FROM <http://kg.socialsecurity.be/resource/annexes/11/>
FROM <http://kg.socialsecurity.be/resource/annexes/21/>
FROM <http://kg.socialsecurity.be/resource/annexes/27/>
FROM <http://kg.socialsecurity.be/resource/annexes/28/>
FROM <http://kg.socialsecurity.be/resource/annexes/31/>
FROM <http://kg.socialsecurity.be/resource/annexes/35/>
FROM <http://kg.socialsecurity.be/resource/annexes/42/>
FROM <http://kg.socialsecurity.be/resource/annexes/43/>
FROM <http://kg.socialsecurity.be/resource/annexes/44/>
FROM <http://kg.socialsecurity.be/resource/annexes/45/>
FROM <http://kg.socialsecurity.be/resource/annexes/46/>
FROM <http://kg.socialsecurity.be/resource/annexes/dmfas02/>
WHERE{
	?s ?p ?o.
}
"""

sparql.setQuery(FETCH_ANNEXES_QUERY)
response = sparql.queryAndConvert().decode("utf-8")

datagraph = open(datagraphFilename, mode="a", encoding="utf-8")
datagraph.write("\n" + response)
datagraph.close()

Creating shape graph


In [ ]:
FETCH_SHAPES_QUERY = """
CONSTRUCT{{
	?s ?p ?o.
}}
FROM <http://kg.socialsecurity.be/resource/shapes/core/{0}/>
{1}
WHERE{{
	?s ?p ?o.
}}
"""

specificGraph = "FROM <http://kg.socialsecurity.be/resource/shapes/" + docType.lower() + "/" + quarter + "/>" if docType.lower() != "core" else ""
sparql.setQuery(FETCH_SHAPES_QUERY.format(quarter, specificGraph))
response = sparql.queryAndConvert().decode("utf-8")

shapegraphFilename = "shapes_tmp.ttl"
shapegraph = open(shapegraphFilename, mode="w", encoding="utf-8")
shapegraph.write(response)
shapegraph.close()

Validate data graph against shape graph with TopBraid


In [ ]:
import os

validationReportFilename = filename.split(".")[0] + "_" + docType.lower() + "_" + quarter + "validation_report.ttl"

os.system('cmd /c "shaclvalidate.bat -datafile {} -shapesfile {} > {}"'.format(datagraphFilename, shapegraphFilename, validationReportFilename))

## Clean up

In [ ]:
os.remove(datagraphFilename)
os.remove(shapegraphFilename)